In [1]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:
from sklearn.datasets import load_iris # Load Iris Data

iris = load_iris() # Creating pd DataFrames

iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
target_df = pd.DataFrame(data= iris.target, columns= ['species'])

def converter(specie):
    if specie == 0:
        return 'setosa'
    elif specie == 1:
        return 'versicolor'
    else:
        return 'virginica'

target_df['species'] = target_df['species'].apply(converter)# Concatenate the DataFrames
iris_df = pd.concat([iris_df, target_df], axis= 1)

In [4]:
# Converting Objects to Numerical dtype
iris_df.drop('species', axis= 1, inplace= True)
target_df = pd.DataFrame(columns= ['species'], data= iris.target)
iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
y= iris_df['sepal length (cm)']

# Splitting the Dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.13, random_state= 101)

# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X_train, y_train)

# Making Predictions
lr.predict(X_test)
pred = lr.predict(X_test)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))

mse = mean_squared_error(y_test, pred)

Mean Squared Error: 0.1123029571353145


In [5]:
shuffle = lambda x: np.random.shuffle(x)

In [6]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
X = X_test.values[:]
Yt_expected = y_test.values[:].reshape(-1, 1)
mse_target = 0.11232375

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('dataset/X.npy',X)
np.save('dataset/Y.npy',Yt_expected)

In [7]:
shuffle(W)
shuffle(b)
print(mean_squared_error(y_test, np.matmul(X_test, W.reshape(-1)) + b.reshape(-1)))

np.save('model_shuffled/W.npy',W)
np.save('model_shuffled/b.npy',b)

0.2016566502876671


In [8]:
Y_expected = np.matmul(X_test, lr.coef_) + lr.intercept_

assert (Y_expected == lr.predict(X_test)).all()

In [9]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = mse_target
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

In [10]:
zkml add_bounty --amount 11.1 --keyfile ./out --walletprivatekey ./keys/.private_key --dataset ./dataset --settings ./settings.json

Hashed inputs: 
14797455496207951391356508759149962584765968173479481191220882411966396840571n
Your Public Key: 
[
  12060908413538201792894054202921076854256908664654936847650046798114054806836n,
  8490429124186248938105519123104739452287602364733895530169747915759012304707n
]
Your Private Key: 
5759055287239188186952624470596478892070755867490418747173631146704844514845n
Current Balance
9029.040027305


In [11]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[
  {
    'publickey-1': '9034578064572512214535301210539208408830664951051263007560019313942236166417',
    'publickey-2': '4995903678092667726400702912124052816223616090560699902572601312415845443763',
    'MSE-Cap': '18406'
  },
  {
    'publickey-1': '6034246696949266198719295105718623837645220054485528158005177132825214698550',
    'publickey-2': '18373206073940188927074635457118967618467836856367301013723120319277640982446',
    'MSE-Cap': '18406'
  }
]


In [12]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]


In [13]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled --dataset ./dataset --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.20165665028766702
... quantized  33041
Mean Squared Error simulated:  0.20177623145943968108
... quantized  33061
Circuit Outputs:
33061
Proof took 18.21 s
ERROR: Invalid proof
An unexpected error occurred:

Error: Could not verify the proof
    at SimpleTaskDefinition.action (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/hardhat.config.js:246:26)
    at Environment._runTaskDefinition (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:217:14)
    at Environment.run (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:129:14)
    at main (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/cli/cli.ts:197:5)


In [14]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --dataset ./dataset  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.11230295713531464
... quantized  18403
Mean Squared Error simulated:  0.11232375022889581895
... quantized  18406
Circuit Outputs:
18406
Proof took 17.852 s
INFO: OK!
Paying 0x2546BcD3c84621e976D8185a91A922aE77ECEc30
With balance
10534.4
Your Public Key: 
[
  18782685461751928671655049660894467121186505774997799430262952740146315877959n,
  3586312028609090491687124451725437750342434223796397850556593341065787359387n
]
Your Private Key: 
15810074267045938560613252922231619211285902817723257628868238586166121631315n
Current Balance
10545.5


In [15]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[
  {
    'publickey-1': '9034578064572512214535301210539208408830664951051263007560019313942236166417',
    'publickey-2': '4995903678092667726400702912124052816223616090560699902572601312415845443763',
    'MSE-Cap': '18406'
  }
]


In [16]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]
